In [4]:
from ch2.data import *

s = session('-v2')
ff_data = statistics(s, FITNESS, FATIGUE)
hr10_data = statistics(s, HR_IMPULSE_10, with_sources=True)
hr10_data.describe()

,HR Impulse / 10s,Src HR Impulse / 10s
count,3.305480e+05,330548.000000
mean,5.382623e-02,134172.745211
std,8.820321e-02,249.381068
min,0.000000e+00,133674.000000
25%,9.182736e-07,133970.000000
50%,1.614737e-02,134214.000000
75%,6.690535e-02,134374.000000
max,1.241608e+00,134666.000000


In [5]:
from ch2.squeal import *

segment_journals = s.query(SegmentJournal).join(Segment).filter(Segment.name == 'San Cristobal').all()
st_data = statistics(s, SEGMENT_TIME, sources=segment_journals)
st_data.describe()

,Segment Time
count,16.000000
mean,1066.450698
std,164.622821
min,917.020896
25%,964.964191
50%,1027.067388
75%,1094.771060
max,1566.123241


In [6]:
import pandas as pd
import datetime as dt
from collections import namedtuple
from ch2.data.impulse import *

# scale of 10 because 10 secs between impulses
model = DecayModel(1e-10, 0, 10, 42 * 24, HR_IMPULSE_10, FITNESS)
hr3600_data = pre_calc(hr10_data, model, target=st_data)
print(hr3600_data.describe())

       Fitness  Impulse / 3600s
count  24678.0     24678.000000
mean       0.0         0.720972
std        0.0         4.599524
min        0.0         0.000000
25%        0.0         0.000000
50%        0.0         0.000000
75%        0.0         0.000000
max        0.0        92.798399


In [7]:
hr3600_data = calc(hr3600_data, model)
print(hr3600_data)

                               Fitness  Impulse / 3600s
2016-07-04 14:00:00+00:00     0.000001     1.000000e-10
2016-07-04 15:00:00+00:00   760.962035     7.609620e+01
2016-07-04 16:00:00+00:00  1146.793225     3.865857e+01
2016-07-04 17:00:00+00:00  1145.656098     0.000000e+00
2016-07-04 18:00:00+00:00  1144.520098     0.000000e+00
2016-07-04 19:00:00+00:00  1143.385224     0.000000e+00
2016-07-04 20:00:00+00:00  1142.251476     0.000000e+00
2016-07-04 21:00:00+00:00  1141.118852     0.000000e+00
2016-07-04 22:00:00+00:00  1139.987351     0.000000e+00
2016-07-04 23:00:00+00:00  1138.856972     0.000000e+00
2016-07-05 00:00:00+00:00  1137.727713     0.000000e+00
2016-07-05 01:00:00+00:00  1136.599575     0.000000e+00
2016-07-05 02:00:00+00:00  1135.472555     0.000000e+00
2016-07-05 03:00:00+00:00  1134.346653     0.000000e+00
2016-07-05 04:00:00+00:00  1133.221867     0.000000e+00
2016-07-05 05:00:00+00:00  1132.098196     0.000000e+00
2016-07-05 06:00:00+00:00  1130.975640     0.000

In [8]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.palettes import Dark2
from bokeh.models import LinearAxis, Range1d

output_notebook()
f = figure(plot_width=500, plot_height=450, x_axis_type='datetime')
#f.extra_y_ranges = {'Speed': Range1d(start=-0.000067, end=0.0014)}  # start by hand to align zero
#f.add_layout(LinearAxis(y_range_name='Speed', axis_label='Speed (arbitrary units)'), 'right')
pt = Dark2[3]
f.line(x=ff_data.index, y=ff_data[FITNESS], color=pt[0], legend=FITNESS)
f.line(x=hr3600_data.index, y=hr3600_data[FITNESS], color=pt[1], legend=f'new {FITNESS}')
#f.circle(x=st_data.index, y=1.0/st_data[SEGMENT_TIME], fill_color=pt[2], size=10, legend='Speed', y_range_name='Speed')
f.legend.location = 'top_left'
show(f)

Loading BokehJS ...

In [9]:
from ch2.data.lib import fit

st_data = st_data.set_index(st_data.index.round('1H'))
data = pre_calc(hr10_data, model, target=st_data)
data = data.join(st_data, how='outer')
data[SPEED] = 1/data[SEGMENT_TIME]

def evaluate(data, model):
    data = calc(data, model)
    data[FITNESS] += model.zero
    return data

data = evaluate(data, model._replace(scale=1))
model = model._replace(scale=(data[SPEED] / data[FITNESS]).dropna().mean())
print(model)
result = fit(SPEED, FITNESS, data, model, evaluate, 'scale', 'period', 'zero', tol=1e-10)
print(result)

DecayModel(start=1e-10, zero=0, scale=1.006986926769798e-06, period=1008, input='HR Impulse / 10s', output='Fitness')
DecayModel(start=1e-10, zero=1.227316366144791e-07, scale=4.7611726435024755e-08, period=6300.872726229902, input='HR Impulse / 10s', output='Fitness')


In [10]:
results = evaluate(data, result)
output_notebook()
f = figure(plot_width=500, plot_height=450, x_axis_type='datetime')
pt = Dark2[3]
# f.line(x=ff_data.index, y=ff_data[FITNESS], color=pt[0], legend=FITNESS)
f.line(x=results.index, y=results[FITNESS], color=pt[1], legend=f'new {FITNESS}')
f.circle(x=st_data.index, y=1.0/st_data[SEGMENT_TIME], fill_color=pt[2], size=10, legend='Speed')
f.legend.location = 'top_left'
show(f)

Loading BokehJS ...

In [11]:
x = (data[FITNESS] - data[SPEED]).dropna()
print(x)
x2 = x * x
print(sum(x2))

2016-07-27 12:00:00+00:00    0.000013
2016-08-25 16:00:00+00:00   -0.000102
2016-09-07 11:00:00+00:00    0.000196
2016-09-09 11:00:00+00:00   -0.000043
2016-12-27 11:00:00+00:00    0.000025
2017-01-18 11:00:00+00:00   -0.000030
2017-02-09 11:00:00+00:00   -0.000108
2017-05-05 10:00:00+00:00    0.000033
2017-07-12 12:00:00+00:00    0.000050
2017-07-14 14:00:00+00:00    0.000022
2017-08-08 12:00:00+00:00   -0.000006
2017-08-10 14:00:00+00:00   -0.000043
2017-08-12 12:00:00+00:00    0.000016
2017-10-04 11:00:00+00:00   -0.000067
2018-01-12 10:00:00+00:00    0.000020
2018-01-26 11:00:00+00:00    0.000023
dtype: float64
7.569606255316785e-08


In [12]:
print(hr10_data)

                           HR Impulse / 10s  Src HR Impulse / 10s
Index                                                            
2016-07-04 14:01:28+00:00          0.000000                133674
2016-07-04 14:01:38+00:00          0.000000                133674
2016-07-04 14:01:48+00:00          0.000000                133674
2016-07-04 14:01:58+00:00          0.000000                133674
2016-07-04 14:02:08+00:00          0.000000                133674
2016-07-04 14:02:18+00:00          0.000000                133674
2016-07-04 14:02:28+00:00          0.000000                133674
2016-07-04 14:02:38+00:00          0.000000                133674
2016-07-04 14:02:48+00:00          0.000000                133674
2016-07-04 14:02:58+00:00          0.000000                133674
2016-07-04 14:03:08+00:00          0.000000                133674
2016-07-04 14:03:18+00:00          0.000000                133674
2016-07-04 14:03:28+00:00          0.000000                133674
2016-07-04